In [2]:
import pandas as pd
import numpy as np
# import category_encoders
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
# from isotree import IsolationForest
import datetime
import gc

DATA_PATH = "dataset/"

# Data Preprocessing.
<INSERT STEPS HERE>
output: train.npy,test.npy

In [3]:
train_df = pd.read_csv(DATA_PATH + 'train.csv')

# Remove outliers
train_df = train_df [ train_df['building_id'] != 1099 ]
train_df = train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

building_df = pd.read_csv(DATA_PATH + 'building_metadata.csv')
weather_df = pd.read_csv(DATA_PATH + 'weather_train.csv')

  Using cached isotree-0.6.1.post2.tar.gz (300 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build isotree


  error: subprocess-exited-with-error
  
  × Building wheel for isotree (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [16 lines of output]
      C:\Users\dublu\AppData\Local\Temp\pip-build-env-ph3eewo1\overlay\Lib\site-packages\setuptools\_distutils\extension.py:134: UserWarning: Unknown Extension options: 'install_requires'
        warnings.warn(msg)
      C:\Users\dublu\AppData\Local\Temp\pip-build-env-ph3eewo1\overlay\Lib\site-packages\setuptools\dist.py:318: InformationOnly: Normalizing '0.6.1-2' to '0.6.1.post2'
        self.metadata.version = self._normalize_version(self.metadata.version)
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\isotree
      copying isotree\__init__.py -> build\lib.win-amd64-cpython-310\isotree
      running build_ext
      Compiling isotree/cpp_interface.pyx because it changed.
      [1/1] Cythonizing

In [4]:
print(train_df.head())
print(building_df.head())
print(weather_df.head())


     building_id  meter            timestamp  meter_reading
103          105      0  2016-01-01 00:00:00        23.3036
104          106      0  2016-01-01 00:00:00         0.3746
105          106      3  2016-01-01 00:00:00         0.0000
106          107      0  2016-01-01 00:00:00       175.1840
107          108      0  2016-01-01 00:00:00        91.2653
   site_id  building_id primary_use  square_feet  year_built  floor_count
0        0            0   Education         7432      2008.0          NaN
1        0            1   Education         2720      2004.0          NaN
2        0            2   Education         5376      1991.0          NaN
3        0            3   Education        23685      2002.0          NaN
4        0            4   Education       116607      1975.0          NaN
   site_id            timestamp  air_temperature  cloud_coverage  \
0        0  2016-01-01 00:00:00             25.0             6.0   
1        0  2016-01-01 01:00:00             24.4            

In [5]:
# Original code from https://www.kaggle.com/aitude/ashrae-missing-weather-data-handling by @aitude

def fill_weather_dataset(weather_df):
    
    # Find Missing Dates
    time_format = "%Y-%m-%d %H:%M:%S"
    start_date = datetime.datetime.strptime(weather_df['timestamp'].min(),time_format)
    end_date = datetime.datetime.strptime(weather_df['timestamp'].max(),time_format)
    total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
    hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]

    missing_hours = []
    for site_id in range(16):
        site_hours = np.array(weather_df[weather_df['site_id'] == site_id]['timestamp'])
        new_rows = pd.DataFrame(np.setdiff1d(hours_list,site_hours),columns=['timestamp'])
        new_rows['site_id'] = site_id
        weather_df = pd.concat([weather_df,new_rows])

        weather_df = weather_df.reset_index(drop=True)           

    # Add new Features
    weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
    weather_df["day"] = weather_df["datetime"].dt.day
    weather_df["week"] = weather_df["datetime"].dt.weekday
    weather_df["month"] = weather_df["datetime"].dt.month
    
    # Reset Index for Fast Update
    weather_df = weather_df.set_index(['site_id','day','month'])

    air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_df.update(air_temperature_filler,overwrite=False)

    # Step 1
    cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    # Step 2
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

    weather_df.update(cloud_coverage_filler,overwrite=False)

    due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_df.update(due_temperature_filler,overwrite=False)

    # Step 1
    sea_level_filler = weather_df.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    # Step 2
    sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

    weather_df.update(sea_level_filler,overwrite=False)

    wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_df.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_df.update(wind_speed_filler,overwrite=False)

    # Step 1
    precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    # Step 2
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

    weather_df.update(precip_depth_filler,overwrite=False)

    weather_df = weather_df.reset_index()
    weather_df = weather_df.drop(['datetime','day','week','month'],axis=1)
        
    return weather_df

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def features_engineering(df):
    
    # Sort by timestamp
    df.sort_values("timestamp")
    df.reset_index(drop=True)
    
    # Add more features
    df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    df["hour"] = df["timestamp"].dt.hour
    df["weekend"] = df["timestamp"].dt.weekday
    holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
                    "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
                    "2017-01-02", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
                    "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
                    "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
                    "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
                    "2019-01-01"]
    df["is_holiday"] = (df.timestamp.isin(holidays)).astype(int)
    df['square_feet'] =  np.log1p(df['square_feet']**0.5)
    
    # Remove Unused Columns
    drop = ["timestamp","sea_level_pressure", "wind_direction", "wind_speed","year_built","floor_count"]
    df = df.drop(drop, axis=1)
    gc.collect()
    
    # Encode Categorical Data
    le = LabelEncoder()
    df["primary_use"] = le.fit_transform(df["primary_use"])
    
    return df

In [6]:
weather_df = fill_weather_dataset(weather_df)

C:\Users\dublu\AppData\Local\Temp\ipykernel_37172\2128334198.py:36: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])
C:\Users\dublu\AppData\Local\Temp\ipykernel_37172\2128334198.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])
C:\Users\dublu\AppData\Local\Temp\ipykernel_37172\2128334198.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])


In [7]:
train_df = reduce_mem_usage(train_df,use_float16=True)
building_df = reduce_mem_usage(building_df,use_float16=True)
weather_df = reduce_mem_usage(weather_df,use_float16=True)

C:\Users\dublu\AppData\Local\Temp\ipykernel_37172\2128334198.py:82: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_datetime(df[col]) or is_categorical_dtype(df[col]):
C:\Users\dublu\AppData\Local\Temp\ipykernel_37172\2128334198.py:82: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_datetime(df[col]) or is_categorical_dtype(df[col]):


Memory usage of dataframe is 757.31 MB


C:\Users\dublu\AppData\Local\Temp\ipykernel_37172\2128334198.py:82: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_datetime(df[col]) or is_categorical_dtype(df[col]):


Memory usage after optimization is: 322.18 MB
Decreased by 57.5%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.9%
Memory usage of dataframe is 9.65 MB
Memory usage after optimization is: 2.60 MB
Decreased by 73.1%


C:\Users\dublu\AppData\Local\Temp\ipykernel_37172\2128334198.py:82: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_datetime(df[col]) or is_categorical_dtype(df[col]):
C:\Users\dublu\AppData\Local\Temp\ipykernel_37172\2128334198.py:82: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_datetime(df[col]) or is_categorical_dtype(df[col]):
C:\Users\dublu\AppData\Local\Temp\ipykernel_37172\2128334198.py:82: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_datetime(df[col]) or is_categorical_dtype(df[col]):
C:\Users\dublu\AppData\Local\Temp\ipykernel_37172\2128334198.py:82: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) 

In [8]:
train_df = train_df.merge(building_df, left_on='building_id',right_on='building_id',how='left')
train_df = train_df.merge(weather_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
del weather_df
gc.collect()

0

In [9]:
train_df = features_engineering(train_df)

In [10]:
target = np.log1p(train_df["meter_reading"])
train = train_df.drop(['meter_reading', 'is_holiday', 'hour', 'weekend'], axis = 1)
del train_df
gc.collect()
train.head()

,building_id,meter,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr
0,105,0,1,0,5.420515,3.800781,0.0,2.400391,0.0
1,106,0,1,0,4.308213,3.800781,0.0,2.400391,0.0
2,106,3,1,0,4.308213,3.800781,0.0,2.400391,0.0
3,107,0,1,0,5.747165,3.800781,0.0,2.400391,0.0
4,108,0,1,0,5.658165,3.800781,0.0,2.400391,0.0


In [11]:
categorical_features = ["building_id", "site_id", "meter", "primary_use"]

In [12]:
import category_encoders

In [13]:
ce = category_encoders.CountEncoder(cols=categorical_features)
ce.fit(train)
train = ce.transform(train)
train.head()

,building_id,meter,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr
0,8784,11706016,553357,8050507,5.420515,3.800781,0.0,2.400391,0.0
1,17564,11706016,553357,8050507,4.308213,3.800781,0.0,2.400391,0.0
2,17564,1264037,553357,8050507,4.308213,3.800781,0.0,2.400391,0.0
3,8784,11706016,553357,8050507,5.747165,3.800781,0.0,2.400391,0.0
4,8784,11706016,553357,8050507,5.658165,3.800781,0.0,2.400391,0.0


In [14]:
N_train = train.shape[0]
for feature in categorical_features:
    train[feature] = train[feature]/N_train
    
train.head()

,building_id,meter,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr
0,0.000442,0.589652,0.027874,0.405518,5.420515,3.800781,0.0,2.400391,0.0
1,0.000885,0.589652,0.027874,0.405518,4.308213,3.800781,0.0,2.400391,0.0
2,0.000885,0.063672,0.027874,0.405518,4.308213,3.800781,0.0,2.400391,0.0
3,0.000442,0.589652,0.027874,0.405518,5.747165,3.800781,0.0,2.400391,0.0
4,0.000442,0.589652,0.027874,0.405518,5.658165,3.800781,0.0,2.400391,0.0


In [15]:
'''%%time
iso = IsolationForest(build_imputer=True, prob_pick_pooled_gain=1, ntry=10)
iso.fit(train)
train = iso.transform(train)'''

'%%time\niso = IsolationForest(build_imputer=True, prob_pick_pooled_gain=1, ntry=10)\niso.fit(train)\ntrain = iso.transform(train)'

In [16]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(train)
train = imp.transform(train)

In [2]:
# store train data and target for easier use
np.save('dataset/train.npy', train)
np.save('dataset/target.npy', target)

NameError: name 'np' is not defined

In [ ]:
del train, target
gc.collect()

0